In [1]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from exp.ProcessData import *
from exp.PytorchModels import *
from exp.LearnerClass import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
from torchvision import transforms
import PIL.Image
from sklearn.metrics import roc_auc_score
import torchvision.transforms.functional as TF
from types import MethodType
import sandesh
import pydicom

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got

In [2]:
params=json_to_parameters('config.json')
num_folds=5
device = device_by_name('Tesla')
#device='cpu'
torch.cuda.set_device(device)

In [3]:
df = pd.read_csv(params.path.data+'full_train.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
transform=transforms.Compose([RandomResizedCropTransform(scale=(0.7, 1.3), ratio=(0.7, 1.3)),
                              RotateTransform(25),
                              RandomFlipTransform(0.5,0.5),
                              RandomChangeMeanStdTransform(30,0.1),
                              CutoutTransform(0.5,0.2),
                              CutoutTransform(0.25,0.3),
                              ResizeTransform(512,512)])

transform_val=transforms.Compose([ResizeTransform(512,512)])


In [ ]:
model_type='tf_efficientnet_b5_ns'
name_tamplate='image_{}'
image_reader=ImageReader(params.path.train)
ds0=ImageDataset(image_reader,df,transform=transform_val)
ds=ImageDataset(image_reader,df,transform=transform)
model_version=1
seed_add=1234
for SEED in [220]:
    for fold in [0]: #range(2,num_folds): 
        torch.manual_seed(SEED+fold+seed_add)
        np.random.seed(SEED+fold+seed_add)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        model = get_model(model_type,output_size=11, feature_size =256,amp=True,pool=False).to(device)
        name=params.model_format.format(model_type,name_tamplate.format(SEED),model_version,fold)
        print (name)
        learner = Learner(model,None,loss_func=None,name=name,scheduler=None,device=device)
        learner.load_model(params.path.models)
#        learner.init_amp()
        learner.model.last_linear=Noop()
        def new_get_x(self,batch):
            return batch[:1] 
        learner.get_x=MethodType(new_get_x, learner)
#         featuress=[learner.predict(ds0,batch_size=64,num_workers=20,do_eval=True)]
        featuress=[]
        sandesh.send({'name':f'finished 1st {SEED} {fold}'})
        for i in notebook.tqdm(range(6)):
#                 learner.model=learner.model.train()
            features=learner.predict(ds,batch_size=64,num_workers=20,do_eval=True)
            featuress.append(features)
            sandesh.send({'name':f'finished {i+1} {SEED} {fold}'})

        features=np.stack(featuress,0)
        with open(params.path.features+(name.split('.')[0]+'tta.pkl'),'wb') as f:
            pickle.dump(features,f,protocol=4)
        sandesh.send({'name':f'saved {features.shape} {SEED} {fold}'})


tf_efficientnet_b5_ns_image_220_v1_s0.pth


In [ ]:
# features=np.stack(featuress,0)
# with open(params.path.features+(name.split('.')[0]+'.pkl'),'wb') as f:
#     pickle.dump(features,f,protocol=4)


In [ ]:
features.shape

In [ ]:
((-600,1500),(100,700),(40,400))
learner.model.wso.conv.weight.detach()[:,0,0,0]
learner.model.wso.conv.bias.detach()
1/learner.model.wso.conv.weight.detach()[:,0,0,0]
(0.5-learner.model.wso.conv.bias.detach())/learner.model.wso.conv.weight.detach()[:,0,0,0]

In [ ]:
((-600,1500),(100,700),(40,400))
learner.model.wso.conv.weight.detach()[:,0,0,0]
learner.model.wso.conv.bias.detach()
1/learner.model.wso.conv.weight.detach()[:,0,0,0]
(0.5-learner.model.wso.conv.bias.detach())/learner.model.wso.conv.weight.detach()[:,0,0,0]

In [ ]:
1/learner.model.wso.conv.weight

In [ ]:
a=ds0.__getitem__(2555)
b=learner.model.wso.conv(a[0][None])
plt.figure(figsize=(12,12))
_=plt.subplot(2,2,1)
_=plt.imshow(b[0].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,2)
_=plt.imshow(b[0][0].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,3)
_=plt.imshow(b[0][1].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,4)
_=plt.imshow(b[0][2].transpose(0,-1).detach().numpy(),cmap='bone')

In [ ]:
a=ds0.__getitem__(2555)
b=learner.model.wso.conv(a[0][None])
plt.figure(figsize=(12,12))
_=plt.subplot(2,2,1)
_=plt.imshow(b[0].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,2)
_=plt.imshow(b[0][0].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,3)
_=plt.imshow(b[0][1].transpose(0,-1).detach().numpy(),cmap='bone')
_=plt.subplot(2,2,4)
_=plt.imshow(b[0][2].transpose(0,-1).detach().numpy(),cmap='bone')